In [17]:
from Crypto.Cipher import AES

In [18]:
def AES_128(k, m):
    alg = AES.new(k, AES.MODE_ECB)
    return alg.encrypt(m)

In [19]:
def davies_meyer(encrypt, l_key, l_message):
    """
    Arguments :
    encrypt : an encryption function
    l_key : length in bytes of the keys for encrypt
    l_message : length in bytes of the messages for encrypt
    Returns :
    A compression function from messages of length l_key + l_message to
    messages of length l_message , defined by using the Davies - Meyer
    construction
    """
    
    def _xor(b1, b2):
        """
        Funcion que retorna el byte de la operacion XOR entre 2 bytes
        """
        
        result = bytearray()
        for b1, b2 in zip(b1, b2):
            result.append(b1 ^ b2)

        return result

    def compression(message):
        """
        Funcion que retorna el XOR entre el mensaje encriptado con u y v, y v
        segun la construccion de davies meyer
        """

        u = message[0:l_key] # utilizamos el tamaño de la llave
        v = message[l_key : l_key + l_message] # el resto es el mensaje
        enc = encrypt(u, v)
        
        return _xor(enc, v)

    return compression

In [20]:
def pad(message, l_block):
    """
    Arguments :
    message : message to be padded
    l_block : length in bytes of the block
    Returns :
    extension of message that includes the length of message
    ( in bytes ) in its last block
    """
    
    N = len(message)
    tail = N % l_block
    to_add = (l_block - tail) % l_block
    if to_add: # si es que existen elementos para rellenar en nuestr bloque
        message += b"\x01" # agregamos un 1 al principio
        message += b"\x00" * (to_add - 1) # rellenamos con 0's hasta el final
    msize = N % (2**l_block)
    message += msize.to_bytes(l_block, "big")

    return message

In [21]:
def merkle_damgard(IV, comp, l_block):
    """
    Arguments :
    IV : initialization vector for a hash function
    comp : compression function to be used in the Merkle - Damgard
    construction
    l_block : length in bytes of the blocks to be used in the Merkle - Damgard
    construction
    Returns :
    A hash function for messages of arbitrary length , defined by using
    the Merkle - Damgard construction
    """
    
    def hash_func(message):
        padded_message = pad(message, l_block)
        nblocks = len(padded_message) // l_block
        Hi = IV #definimos el vector de inicio
        for i in range(nblocks): # encriptamos por bloque
            mi = padded_message[i*l_block:(i+1)*l_block]
            Hi = comp(mi + Hi)
        return Hi
    return hash_func

# Tests

In [29]:
if __name__ == "__main__": 

    compresion = davies_meyer(AES_128, 16, 16)
    
    hash = merkle_damgard(b"0123456789012345", compresion, 16)
    
    h1 = hash(b"Este es un mensaje de prueba para la tarea 2")
    h2 = hash(b"Este es un segundo mensaje de prueba para la tarea 2")
    
    print(h1)
    print(h2)

bytearray(b'\xe4MC\xd5\xc8\xbfv\xe3a\xf1\xf6w\x94\xa5\xac\x9b')
bytearray(b'As\xa0#$\x19\xb5K.d\xc4\xeb7]\xf7H')
